In [189]:
import requests, time, os
import pandas as pd
from pandas.io.json import json_normalize
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from functools import reduce
import operator

In [99]:
client_credentials_manager = SpotifyClientCredentials()
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [105]:
def spotify_playlist_tracks(playlist_id):
    
    offset = 0
    track_listings = []
    
    touch = sp.playlist_tracks(playlist_id, fields="track(name%2C%20id%2C%20album(name))", 
                                    limit = 1, offset = 0, market='ES')
    num_listings = touch['total']

    for ii in range(0,num_listings, 100):
        tracks = sp.playlist_tracks(playlist_id, fields="track(name%2C%20id%2C%20album(name))", 
                                    limit = 100, offset = ii, market='ES')
        while tracks:
            for i, track in enumerate(tracks['items']):
                track_listings.append([tracks['items'][i]['track']['artists'][0]['name'],
                                     tracks['items'][i]['track']['album']['name'], tracks['items'][i]['track']['name'],
                                     tracks['items'][i]['track']['id']])

            if tracks['next']:
                tracks = sp.next(tracks)
            else:
                tracks = None
                
        time.sleep(0.3) #To make sure we don't abuse spotify's API terms
        
    return track_listings

In [106]:
playlist_id = '7htu5ftbLBRFAwiuHVcUAg'
track_listings = spotify_playlist_tracks(playlist_id)

In [110]:
columns = ['artist', 'album', 'track', 'track_id']

track_df = pd.DataFrame(track_listings, columns = columns)

In [89]:
# track_df.to_csv('track_df.csv') #save point

In [109]:
#track_df = pd.read_csv('track_df.csv') #load if needed

In [115]:
print(track_df.shape)
track_df.head()

(9988, 4)


,artist,album,track,track_id
0,Lorde,Pure Heroine,Ribs,2MvvoeRt8NcOXWESkxWn3g
1,Clairo,Immunity,Bags,6UFivO2zqqPFPoQYsEMuCc
2,The Script,The Script,The Man Who Can't Be Moved,4Musyaro0NM5Awx8b5c627
3,Adele,21,Rolling in the Deep,1eq1wUnLVLg4pdEfx9kajC
4,Bruno Mars,Doo-Wops & Hooligans,Grenade,4lLtanYk6tkMvooU0tWzG8


In [155]:
track_ids = list(track_df['track_id'][0:5])
json_normalize(sp.audio_features(track_ids))

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.511,0.472,4,-9.277,1,0.0910,0.534,0.614,0.1100,0.0399,127.978,audio_features,2MvvoeRt8NcOXWESkxWn3g,spotify:track:2MvvoeRt8NcOXWESkxWn3g,https://api.spotify.com/v1/tracks/2MvvoeRt8NcO...,https://api.spotify.com/v1/audio-analysis/2Mvv...,258969,4
1,0.742,0.546,1,-7.694,1,0.0315,0.172,0.380,0.1150,0.8680,104.996,audio_features,6UFivO2zqqPFPoQYsEMuCc,spotify:track:6UFivO2zqqPFPoQYsEMuCc,https://api.spotify.com/v1/tracks/6UFivO2zqqPF...,https://api.spotify.com/v1/audio-analysis/6UFi...,260520,4
2,0.609,0.629,10,-5.024,1,0.0264,0.425,0.000,0.0978,0.3250,99.955,audio_features,4Musyaro0NM5Awx8b5c627,spotify:track:4Musyaro0NM5Awx8b5c627,https://api.spotify.com/v1/tracks/4Musyaro0NM5...,https://api.spotify.com/v1/audio-analysis/4Mus...,241467,4
3,0.730,0.770,8,-5.114,1,0.0298,0.138,0.000,0.0473,0.5070,104.948,audio_features,1eq1wUnLVLg4pdEfx9kajC,spotify:track:1eq1wUnLVLg4pdEfx9kajC,https://api.spotify.com/v1/tracks/1eq1wUnLVLg4...,https://api.spotify.com/v1/audio-analysis/1eq1...,228093,4
4,0.706,0.558,2,-7.237,0,0.0593,0.146,0.000,0.1180,0.2270,110.443,audio_features,4lLtanYk6tkMvooU0tWzG8,spotify:track:4lLtanYk6tkMvooU0tWzG8,https://api.spotify.com/v1/tracks/4lLtanYk6tkM...,https://api.spotify.com/v1/audio-analysis/4lLt...,223253,4


In [192]:
def spotify_track_features(track_ids):
    
    track_features = []
    
    for ii in range(0, len(track_ids), 100):
        if ii <= (len(track_ids)-100):
            track_features.append(sp.audio_features(track_ids[ii:ii+100]))
        else:
            track_features.append(sp.audio_features(track_ids[ii:]))           
            
        time.sleep(0.3)
        
    track_features = reduce(operator.add, track_features)
    track_features_df = json_normalize(track_features)
        
    return track_features_df

In [194]:
tracks = list(track_df['track_id'])
track_features_df = spotify_track_features(tracks)

In [197]:
print(track_features_df.shape)
track_features_df.head()

(9988, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.511,0.472,4,-9.277,1,0.0910,0.534,0.614,0.1100,0.0399,127.978,audio_features,2MvvoeRt8NcOXWESkxWn3g,spotify:track:2MvvoeRt8NcOXWESkxWn3g,https://api.spotify.com/v1/tracks/2MvvoeRt8NcO...,https://api.spotify.com/v1/audio-analysis/2Mvv...,258969,4
1,0.742,0.546,1,-7.694,1,0.0315,0.172,0.380,0.1150,0.8680,104.996,audio_features,6UFivO2zqqPFPoQYsEMuCc,spotify:track:6UFivO2zqqPFPoQYsEMuCc,https://api.spotify.com/v1/tracks/6UFivO2zqqPF...,https://api.spotify.com/v1/audio-analysis/6UFi...,260520,4
2,0.609,0.629,10,-5.024,1,0.0264,0.425,0.000,0.0978,0.3250,99.955,audio_features,4Musyaro0NM5Awx8b5c627,spotify:track:4Musyaro0NM5Awx8b5c627,https://api.spotify.com/v1/tracks/4Musyaro0NM5...,https://api.spotify.com/v1/audio-analysis/4Mus...,241467,4
3,0.730,0.770,8,-5.114,1,0.0298,0.138,0.000,0.0473,0.5070,104.948,audio_features,1eq1wUnLVLg4pdEfx9kajC,spotify:track:1eq1wUnLVLg4pdEfx9kajC,https://api.spotify.com/v1/tracks/1eq1wUnLVLg4...,https://api.spotify.com/v1/audio-analysis/1eq1...,228093,4
4,0.706,0.558,2,-7.237,0,0.0593,0.146,0.000,0.1180,0.2270,110.443,audio_features,4lLtanYk6tkMvooU0tWzG8,spotify:track:4lLtanYk6tkMvooU0tWzG8,https://api.spotify.com/v1/tracks/4lLtanYk6tkM...,https://api.spotify.com/v1/audio-analysis/4lLt...,223253,4


In [198]:
# track_features_df.to_csv('track_features_df.csv') # save point

In [ ]:
#track_features_df = pd.read_csv('track_features_df.csv') # load if necessary